In [4]:
from transformers import VitsModel, AutoTokenizer
import torch
import scipy.io.wavfile as wavfile

In [5]:
# 1. เช็คและกำหนด Device (ถ้ามี GPU จะใช้ cuda ถ้าไม่มีจะกลับไปใช้ cpu)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [6]:
# 2. โหลด Model และ Tokenizer
model_name = "facebook/mms-tts-tha"
model = VitsModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [7]:
# 3. ย้าย Model ไปยัง GPU
model.to(device)

VitsModel(
  (text_encoder): VitsTextEncoder(
    (embed_tokens): Embedding(71, 192)
    (encoder): VitsEncoder(
      (layers): ModuleList(
        (0-5): 6 x VitsEncoderLayer(
          (attention): VitsAttention(
            (k_proj): Linear(in_features=192, out_features=192, bias=True)
            (v_proj): Linear(in_features=192, out_features=192, bias=True)
            (q_proj): Linear(in_features=192, out_features=192, bias=True)
            (out_proj): Linear(in_features=192, out_features=192, bias=True)
          )
          (dropout): Dropout(p=0.1, inplace=False)
          (layer_norm): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
          (feed_forward): VitsFeedForward(
            (conv_1): Conv1d(192, 768, kernel_size=(3,), stride=(1,))
            (conv_2): Conv1d(768, 192, kernel_size=(3,), stride=(1,))
            (dropout): Dropout(p=0.1, inplace=False)
            (act_fn): ReLU()
          )
          (final_layer_norm): LayerNorm((192,), eps=1e-05, eleme

In [8]:
text = """ไอส์พบว่าตัวเองอยู่ในความมืด ไม่รู้ว่าตัวเองอยู่ที่ไหนหรือทำไมถึงมาอยู่ที่นี่ ขณะที่เธอหันมองไปรอบๆ เธอตระหนักได้ว่าตัวเองได้เปลี่ยนกลับไปเป็นเด็กน้อยอีกครั้ง และเธอรู้สึกโดดเดี่ยว ทันใดนั้นของเล่นในวัยเด็กของเธอก็ปรากฏตัวขึ้นเป็นเพื่อนๆ ได้แก่ ตุ๊กตาพี่สาว หมาป่าตุ๊กตาหมีนุ่ม ทหารหุ่นยนต์ เอลฟ์แสง และทหารรูปแมว ทั้งหมดต่างคอยให้กำลังใจเธอ ด้วยความกล้าหาญ ไอส์ประกาศตัวเองเป็นนักผจญภัย หยิบดาบและโล่ของเล่นขึ้นมา แล้วเริ่มต้นการผจญภัยพร้อมกับเพื่อนๆ

พวกเขาเผชิญหน้ากับเขาวงกตสลัวที่เต็มไปด้วยสัตว์ประหลาด แต่ไอส์ไม่รู้สึกหวาดกลัวเลย ด้วยความช่วยเหลือจากเพื่อนตัวเล็กๆ ของเธอ พวกเขาเอาชนะสัตว์ประหลาดไปทีละตัว เหล่าภูตผีส่องแสงนำทางพวกเขาด้วยเวทมนตร์สีสันสดใส ไอส์ฟันดาบของเธอและสามารถเอาชนะสัตว์ประหลาดได้สำเร็จ นำเพื่อนๆ ของเธอไปร้องเพลงและเต้นรำอย่างมีความสุขขณะที่พวกเขาเดินลึกลงไปในเขาวงกต

พวกเขาลงไปลึกลงไปอีกหกสิบชั้น จนกระทั่งมีลมเย็นพัดมา บางสิ่งเปียกและเหนียวๆ ตกกระเด็นใส่ไอส์ และเมื่อเธอเปิดตาขึ้น เธอก็พบว่าความเงียบสงัด เมื่อหันกลับไป เธอรู้สึกหวาดกลัวเมื่อเห็นน้ำพุสีแดงสดที่เพื่อนๆ ของเธอนอนอยู่ด้วยท่าทางบิดเบี้ยวและบาดเจ็บ แขนขาของพวกเขาผิดรูปไปหมด และบางคนจนจำไม่ได้ สารอุ่นๆ บนตัวไอส์ปรากฏว่าเป็นเลือด—เลือดของเพื่อนๆ ของเธอ"""


In [9]:
# 4. เตรียม Input และย้าย Tensors ไปยัง GPU ด้วย .to(device)
inputs = tokenizer(text, return_tensors="pt").to(device)

In [10]:
# 5. สร้างเสียง (Inference)
with torch.no_grad():
    # ผลลัพธ์ที่ได้จะอยู่บน GPU
    output = model(**inputs).waveform

In [11]:
# 6. บันทึกเป็นไฟล์
# ต้องย้ายข้อมูลกลับมาที่ CPU และแปลงเป็น numpy ก่อนบันทึก
sampling_rate = model.config.sampling_rate
audio_data = output[0].cpu().numpy() 

wavfile.write("output.wav", rate=sampling_rate, data=audio_data)
print("Saved output.wav")

Saved output.wav


In [ ]:
from transformers import VitsModel, AutoTokenizer
import torch

# โหลด Model และ Tokenizer
model_name = "facebook/mms-tts-tha"
model = VitsModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

text = "สวัสดีครับ ยินดีที่ได้รู้จัก วันนี้อากาศดีมากเลยนะครับ"

# เตรียม Input
inputs = tokenizer(text, return_tensors="pt")

# สร้างเสียง (Inference)
with torch.no_grad():
    output = model(**inputs).waveform

# บันทึกเป็นไฟล์ .wav หรือเล่นเสียง
import scipy.io.wavfile as wavfile

# อัตราสุ่มสัญญาณ (Sampling Rate) ของ MMS คือ 16000 Hz
wavfile.write("output.wav", rate=model.config.sampling_rate, data=output[0].cpu().numpy())

c:\Users\aungl\anaconda3\envs\ai_work\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
text = """ไอส์พบว่าตัวเองอยู่ในความมืด ไม่รู้ว่าตัวเองอยู่ที่ไหนหรือทำไมถึงมาอยู่ที่นี่ ขณะที่เธอหันมองไปรอบๆ เธอตระหนักได้ว่าตัวเองได้เปลี่ยนกลับไปเป็นเด็กน้อยอีกครั้ง และเธอรู้สึกโดดเดี่ยว ทันใดนั้นของเล่นในวัยเด็กของเธอก็ปรากฏตัวขึ้นเป็นเพื่อนๆ ได้แก่ ตุ๊กตาพี่สาว หมาป่าตุ๊กตาหมีนุ่ม ทหารหุ่นยนต์ เอลฟ์แสง และทหารรูปแมว ทั้งหมดต่างคอยให้กำลังใจเธอ ด้วยความกล้าหาญ ไอส์ประกาศตัวเองเป็นนักผจญภัย หยิบดาบและโล่ของเล่นขึ้นมา แล้วเริ่มต้นการผจญภัยพร้อมกับเพื่อนๆ

พวกเขาเผชิญหน้ากับเขาวงกตสลัวที่เต็มไปด้วยสัตว์ประหลาด แต่ไอส์ไม่รู้สึกหวาดกลัวเลย ด้วยความช่วยเหลือจากเพื่อนตัวเล็กๆ ของเธอ พวกเขาเอาชนะสัตว์ประหลาดไปทีละตัว เหล่าภูตผีส่องแสงนำทางพวกเขาด้วยเวทมนตร์สีสันสดใส ไอส์ฟันดาบของเธอและสามารถเอาชนะสัตว์ประหลาดได้สำเร็จ นำเพื่อนๆ ของเธอไปร้องเพลงและเต้นรำอย่างมีความสุขขณะที่พวกเขาเดินลึกลงไปในเขาวงกต

พวกเขาลงไปลึกลงไปอีกหกสิบชั้น จนกระทั่งมีลมเย็นพัดมา บางสิ่งเปียกและเหนียวๆ ตกกระเด็นใส่ไอส์ และเมื่อเธอเปิดตาขึ้น เธอก็พบว่าความเงียบสงัด เมื่อหันกลับไป เธอรู้สึกหวาดกลัวเมื่อเห็นน้ำพุสีแดงสดที่เพื่อนๆ ของเธอนอนอยู่ด้วยท่าทางบิดเบี้ยวและบาดเจ็บ แขนขาของพวกเขาผิดรูปไปหมด และบางคนจนจำไม่ได้ สารอุ่นๆ บนตัวไอส์ปรากฏว่าเป็นเลือด—เลือดของเพื่อนๆ ของเธอ"""

# เตรียม Input
inputs = tokenizer(text, return_tensors="pt")

# สร้างเสียง (Inference)
with torch.no_grad():
    output = model(**inputs).waveform

# บันทึกเป็นไฟล์ .wav หรือเล่นเสียง
import scipy.io.wavfile as wavfile

# อัตราสุ่มสัญญาณ (Sampling Rate) ของ MMS คือ 16000 Hz
wavfile.write("output2.wav", rate=model.config.sampling_rate, data=output[0].cpu().numpy())

KeyboardInterrupt: 